In [43]:
from keras.src.models import Sequential
from keras.src.layers import Dense, Flatten, Conv2D, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.src.optimizers import Adam
from keras.src.callbacks import EarlyStopping
import os
import keras
import matplotlib.pyplot as plt
from keras.src.applications.resnet import ResNet50
import numpy as np

In [4]:
data_path = 'data'
dirs = os.listdir(data_path)

In [7]:
train, validation = keras.utils.image_dataset_from_directory(directory='data', 
                                                labels='inferred',
                                                label_mode='int',
                                                image_size=(256,256),
                                                batch_size=64,
                                                seed=2009,
                                                validation_split=0.15,
                                                subset='both',
                                                verbose=True)

Found 30965 files belonging to 7 classes.
Using 26321 files for training.
Using 4644 files for validation.


In [8]:
train_set = train.take(25_005)
test_set = train.skip(25_005)

In [16]:
# my neural network
models = Sequential(
    layers = [
        Input(shape=(256, 256, 3), batch_size=64),
        Conv2D(4, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(4, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(8, (5, 5), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(8, (5, 5), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(16, (3, 3), activation='relu'),
        AveragePooling2D((2, 2)),
        Flatten(),
        Dropout(0.3),
        #Dense(512, activation='relu'),
        #Dropout(0.3),
        #Dense(256, activation='relu'),
        #Dropout(0.3),
        Dense(512, activation='relu'),
        Dense(len(dirs), activation='softmax')
    ],
    name='softmax_model_1'
)
print("network created")

network created


In [36]:
# pre-trained neural network
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False
models = Sequential(
    layers = [
        base_model,
        Flatten(),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.5), # era 0.3
        Dense(32, activation='relu'),
        Dense(len(dirs), activation='softmax')
    ],
    name = 'final_model'
)


In [37]:
# Compiling the model
models.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("model compiled")

model compiled


In [40]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
n_epoche = 100

In [41]:
models.summary()

Model: "final_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 131072)         │       524,288 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,001,047 (286.11 MB)

 Trainable params: 17,050,311 (65.04 MB)

 Non-trainable params: 23,850,112 (90.98 MB)

 Optimizer params: 34,100,624 (130.08 MB)

In [42]:
# Fitting the model
history = models.fit(train_set, epochs=n_epoche, validation_data=(validation), batch_size=64, callbacks=[early_stopping])
print("model fitted")

Epoch 1/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 537s 1s/step - accuracy: 0.4093 - loss: 1.5296 - val_accuracy: 0.6652 - val_loss: 0.9403
Epoch 2/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 532s 1s/step - accuracy: 0.6116 - loss: 1.0828 - val_accuracy: 0.7052 - val_loss: 0.8342
Epoch 3/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 542s 1s/step - accuracy: 0.6970 - loss: 0.8592 - val_accuracy: 0.7186 - val_loss: 0.8078
Epoch 4/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 549s 1s/step - accuracy: 0.7518 - loss: 0.7053 - val_accuracy: 0.7261 - val_loss: 0.8040
Epoch 5/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 541s 1s/step - accuracy: 0.8008 - loss: 0.5792 - val_accuracy: 0.7250 - val_loss: 0.8237
Epoch 6/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 533s 1s/step - accuracy: 0.8282 - loss: 0.5034 - val_accuracy: 0.7308 - val_loss: 0.8322
Epoch 7/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 533s 1s/step - accuracy: 0.8563 - loss: 0.4249 - val_accuracy: 0.7306 - val_loss: 0.8586
model fitted


In [48]:
# prediction on test set
predictions = models.predict(test_set, batch_size=64)

c:\Users\mirko\anaconda3\envs\deepLearning\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


ValueError: math domain error

In [49]:
models.save('first_cnn_resnet50.keras')

In [ ]:
# pre trained neural network: ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
models = Sequential(
    layers = [
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(len(dirs), activation='softmax')
    ],
    name='resnet'
)

In [ ]:
my_model = keras.saving.load_model(r'C:\Users\mirko\Projects\deepLearning\30_epoche_soft_max_model_60vacc.keras', custom_objects=None, compile=True, safe_mode=True)

In [ ]:
# plotting the accuracy and the loss
xx = np.arange(1, 31, 1)
plt.subplot(2,1,1)
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.plot(xx, history['loss'], c='red')
plt.plot(xx, history['val_loss'], c='orange')
plt.subplot(2,2,1)
plt.plot(xx, history['accuracy'], c='green')
# plt.plot(xx, history['root_mean_squared_error'], c='red')
# plt.plot(xx, history['val_root_mean_squared_error'], c='orange')
# plt.xlabel('Epoche')
# plt.ylabel('RMSE')
plt.show()